In [376]:
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la conexión con SQL Server usando SQLAlchemy
server = '127.0.0.1'
database = 'Actividad_ETL'
username = 'sa'
password = '12345678'

# Crear la URL de conexión
conn_str = f"mssql+pymssql://{username}:{password}@{server}/{database}"

# Crear el motor de conexión
engine = create_engine(conn_str)

# Ejecutar la consulta SQL y cargar los datos en un DataFrame
query = "SELECT * FROM dbo.tabla_etl_nueva1"
df = pd.read_sql(query, engine)

df.info()  # Tipos de datos, valores nulos

# Renombrar columnas: eliminar espacios y caracteres especiales
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
# Porcentaje de valores nulos por columna
print(df.isnull().sum().to_frame(name="Valores Nulos").assign(Porcentaje=lambda x: (x["Valores Nulos"] / len(df)) * 100))

numerical_columns = ["puntaje_global_icfes", "a_global", "a_lectura_critica", 
                     "a_matematicas", "a_sociales_y_ciudadanas", 
                     "a_ciencias_naturales", "a_ingles", "edad"]

# Identificadores (se mantienen como texto)
identifier_columns = ["codigo_estudiante", "registro_snp", "snp"]

# Aplicar conversiones
df[numerical_columns] = df[numerical_columns].astype("float64")
df[identifier_columns] = df[identifier_columns].astype("object")
df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).astype("category")

# Verificación
df.info()

import pandas as pd

# Ajustar la visualización para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ver las primeras 10 filas
print(df.head(10))

# Reemplazar únicamente los valores 0 en 'edad' con la mediana de la población
mediana_edad = df["edad"].median()
df.loc[df["edad"] == 0, "edad"] = mediana_edad

# Verificar que ya no haya valores nulos
print(df.isnull().sum())  # Debería mostrar solo ceros

import pandas as pd

# Listar las columnas del DataFrame
columnas_presentes = df.columns
print(columnas_presentes)

# Convertir las columnas a string para evitar problemas con categorías
coincidencias = (df["registro_snp"].astype(str) == df["snp"].astype(str)).sum()
diferencias = (df["registro_snp"].astype(str) != df["snp"].astype(str)).sum()

# Mostrar los resultados
print(f"Registros donde 'registro_snp' y 'snp' coinciden: {coincidencias}")
print(f"Registros donde 'registro_snp' y 'snp' son diferentes: {diferencias}")

import pandas as pd

# Convertir las columnas a tipo numérico para evitar problemas de comparación
df["puntaje_global_icfes"] = pd.to_numeric(df["puntaje_global_icfes"], errors="coerce")
df["a_global"] = pd.to_numeric(df["a_global"], errors="coerce")

# Verificar si las columnas son idénticas en todos los registros
if (df["puntaje_global_icfes"] == df["a_global"]).all():
    df.drop(columns=["a_global"], inplace=True)
    print("Las columnas son idénticas. 'a_global' ha sido eliminada.")
else:
    print("Las columnas tienen diferencias, no se ha eliminado ninguna.")


import pandas as pd

# Ajustar la visualización para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ver las primeras 10 filas
print(df.head(10))

# Convertimos las columnas categóricas a string antes de reemplazar valores
df["estado_civil"] = df["estado_civil"].astype(str).replace(
    {"No registra": "No Registra", "Sin Registro": "No Registra", "Unión libre": "Unión Libre"}
).astype("category")

df["colegio_sector"] = df["colegio_sector"].astype(str).replace(
    {"NO REGISTRA": "No Registra", "SIN CLASIFICACION": "Sin Clasificación"}
).astype("category")

df["colegio_clasificacion"] = df["colegio_clasificacion"].astype(str).replace(
    {"NO REGISTRA": "No Registra", 
     "SIN CLASIFICACION": "Sin Clasificación",
     "SIN CLASIFICACIÓN": "Sin Clasificación"}  # Normaliza acentos
).astype("category")


import pandas as pd

# Ajustar la visualización para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ver las primeras 10 filas
print(df.head(10))

df['codigo_estudiante'] = pd.to_numeric(df['codigo_estudiante'].astype(str), errors='coerce').astype('Int64')

print(df['codigo_estudiante'].head(10))  # Ver los primeros 10 valores print(df['codigo_estudiante'].isnull().sum())  # Contar valores nulos print(df.dtypes)  # Revisar el tipo de dato

# Convertir columnas de texto (object) a mayúsculas
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.upper()

# Convertir valores de las columnas categóricas a mayúsculas sin cambiar las categorías
columnas_categoricas = df.select_dtypes(include=['category']).columns
for col in columnas_categoricas:
    # Convertir los valores a mayúsculas sin cambiar las categorías
    df[col] = df[col].cat.codes.map(lambda x: df[col].cat.categories[x].upper())

# Verificar que los datos estén correctos
print(df.head(10))

#Carga de datos transformados a SQL
df.to_sql("TRANSFORMACION", engine, if_exists="replace", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11729 entries, 0 to 11728
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   codigo_estudiante              11729 non-null  float64
 1   Registro_SNP                   11729 non-null  object 
 2   Periodo_Presentacion1          11729 non-null  object 
 3   Periodo_Presentacion           11729 non-null  object 
 4   Puntaje_Global_ICFES           11729 non-null  float64
 5   A_Global                       11729 non-null  float64
 6   A_Lectura_Critica              11729 non-null  float64
 7   A_Matematicas                  11729 non-null  float64
 8   A_Sociales_y_Ciudadanas        11729 non-null  float64
 9   A_Ciencias_Naturales           11729 non-null  float64
 10  A_Ingles                       11729 non-null  float64
 11  A_Nivel_Ingles                 11729 non-null  object 
 12  Periodo_Academico              11727 non-null 

16